In [1]:
import holidays
import pandas as pd
import pyspark.sql.functions as fn
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('garbage').getOrCreate()
sf_large = spark.read.csv('../data/unzip/san_fran_waste_large.csv', header=True)
sf_large.cache()

DataFrame[CaseID: string, Opened: string, Closed: string, Updated: string, Status: string, Status Notes: string, Responsible Agency: string, Category: string, Request Type: string, Request Details: string, Address: string, Street: string, Supervisor District: string, Neighborhood: string, Police District: string, Latitude: string, Longitude: string, Point: string, Source: string, Media URL: string]

In [3]:
sf_large.printSchema()

root
 |-- CaseID: string (nullable = true)
 |-- Opened: string (nullable = true)
 |-- Closed: string (nullable = true)
 |-- Updated: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Notes: string (nullable = true)
 |-- Responsible Agency: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Request Type: string (nullable = true)
 |-- Request Details: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- Supervisor District: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Police District: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Point: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Media URL: string (nullable = true)



In [4]:
sf_large.count(), len(sf_large.columns)

(994422, 20)

In [5]:
sample = sf_large.sample(fraction=.01, seed=242)
sample.toPandas()

,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Street,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source,Media URL
0,12702989,07/28/2020 11:44:00 AM,None,07/28/2020 06:25:04 PM,Open,accepted,DPW Ops Queue,Street and Sidewalk Cleaning,Bulky Items,Refrigerator,Intersection of KEITH ST and PALOU AVE,KEITH ST,10,Bayview,BAYVIEW,37.731857299805,-122.387168884277,"(37.7318573, -122.38716888)",Integrated Agency,None
1,12705207,07/28/2020 05:59:15 PM,None,07/28/2020 06:03:21 PM,Open,accepted,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Furniture,"2454 GREAT HWY, SAN FRANCISCO, CA, 94116",GREAT HWY,4,Outer Sunset,TARAVAL,37.74065833,-122.506568662638,"(37.74065833, -122.50656866)",Mobile/Open311,http://mobile311.sfgov.org/reports/12705207/ph...
2,12705304,07/28/2020 06:25:00 PM,None,07/28/2020 06:55:03 PM,Open,accepted,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,"3000 MARKET ST, SAN FRANCISCO, CA, 94114",MARKET ST,8,Eureka Valley,MISSION,37.7611707,-122.4410268,"(37.7611707, -122.4410268)",Web,None
3,12703351,07/28/2020 12:38:00 PM,07/28/2020 01:18:00 PM,07/28/2020 01:18:00 PM,Closed,Case Resolved,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,"1401 UTAH ST, SAN FRANCISCO, CA, 94110",UTAH ST,10,Potrero Hill,MISSION,37.75158593,-122.40507703,"(37.75158593, -122.40507703)",Mobile/Open311,http://mobile311.sfgov.org/reports/12703351/ph...
4,12705318,07/28/2020 06:30:00 PM,None,07/28/2020 06:55:02 PM,Open,in progress,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,"3564 18TH ST, SAN FRANCISCO, CA, 94110",18TH ST,8,Mission Dolores,MISSION,37.761959205528,-122.422836029721,"(37.76195921, -122.42283603)",Mobile/Open311,http://mobile311.sfgov.org/reports/12705318/ph...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9864,4884137,07/06/2015 10:58:35 AM,07/07/2015 08:48:43 AM,07/07/2015 08:48:43 AM,Closed,Case Completed - resolved: 10/98,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,"170 13TH ST, SAN FRANCISCO, CA, 94103",13TH ST,6,South of Market,MISSION,37.769852,-122.416496,"(37.769852, -122.416496)",Mobile/Open311,None
9865,4889485,07/07/2015 04:34:08 PM,07/08/2015 11:00:00 AM,07/08/2015 11:00:00 AM,Closed,Case Completed - resolved: Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Boxed or Bagged Items,"277 MIRAMAR AVE, SAN FRANCISCO, CA, 94112",MIRAMAR AVE,7,Ingleside,TARAVAL,37.724088902575,-122.458329203482,"(37.7240889, -122.4583292)",Phone,None
9866,4889578,07/07/2015 04:49:57 PM,07/08/2015 10:01:44 AM,07/08/2015 10:01:44 AM,Closed,Case Completed - resolved: Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Electronics,"2500 18TH ST, SAN FRANCISCO, CA, 94110",18TH ST,10,Mission,MISSION,37.762157,-122.407503,"(37.762157, -122.407503)",Mobile/Open311,http://mobile311.sfgov.org/reports/4889578/photos
9867,4889774,07/07/2015 05:40:03 PM,07/08/2015 09:58:57 AM,07/08/2015 09:58:57 AM,Closed,Case Completed - resolved: WASTE NOT FOUND,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Refrigerator,"170 OTIS ST, SAN FRANCISCO, CA, 94103",OTIS ST,6,South of Market,SOUTHERN,37.770843505859,-122.42066192627,"(37.77084351, -122.42066193)",Integrated Agency,None


In [6]:
sf = spark.createDataFrame(sample.collect())
sf.describe().toPandas()

,summary,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,...,Address,Street,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source,Media URL
0,count,9869,9857,9829,9857,9857,9856,9827,9827,9827,...,9827,9799,9799,9791,9776,9796,9796,9796,9796,5042
1,mean,9245731.620598413,None,None,None,None,None,None,6.612903225806452,None,...,37.77030109982187,-122.41399955347336,6.388160589922163,7.0,None,37.66456401583915,-122.03536821937786,-122.39947285311075,None,None
2,stddev,2245595.5811046506,None,None,None,None,None,None,2.5255681783820276,None,...,0.01912836916405582,0.024828510139326986,2.9924085616293588,2.0,None,1.943641827221629,7.079056311123574,0.01109086703015751,None,None
3,min,Add'l Notes:,01/01/2016 04:33:24 PM,01/01/2016 09:37:09 AM,01/01/2016 09:37:09 AM,Boxed or Bagged Items,"""Case Resolved - PRIVATE PROPERTY ...","THANKS.""",1,Automotive,...,"1 6TH ST, SAN FRANCISCO, CA, 94103",-122.371940603594,"(37.7258359, -122.378517)",10,BAYVIEW,0,-122.36857526947,"(0.0, 0.0)","(37.7651133, -122.39776219)",Mobile/Open311
4,max,"cleared""",SFMTA - Parking Enforcement - G,Street and Sidewalk Cleaning,Transit_shelter_platform,Transit_shelter_platform,sent,US Postal Service Maintenance Queue,Street and Sidewalk Cleaning,Transit_shelter_platform,...,"Washington Square, , SAN FRANCISCO, CA, 94133",ZOE ST,TERRY A FRANCOIS BLVD,Westwood Park,http://mobile311.sfgov.org/reports/9855733/photos,SOUTHERN,37.780536501704,-122.41231393185,Web,https://twitter.com/sfplogga/status/1074186668...


In [7]:
sf.limit(20).toPandas()

,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Street,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source,Media URL
0,12702989,07/28/2020 11:44:00 AM,None,07/28/2020 06:25:04 PM,Open,accepted,DPW Ops Queue,Street and Sidewalk Cleaning,Bulky Items,Refrigerator,Intersection of KEITH ST and PALOU AVE,KEITH ST,10,Bayview,BAYVIEW,37.731857299805,-122.387168884277,"(37.7318573, -122.38716888)",Integrated Agency,None
1,12705207,07/28/2020 05:59:15 PM,None,07/28/2020 06:03:21 PM,Open,accepted,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Furniture,"2454 GREAT HWY, SAN FRANCISCO, CA, 94116",GREAT HWY,4,Outer Sunset,TARAVAL,37.74065833,-122.506568662638,"(37.74065833, -122.50656866)",Mobile/Open311,http://mobile311.sfgov.org/reports/12705207/ph...
2,12705304,07/28/2020 06:25:00 PM,None,07/28/2020 06:55:03 PM,Open,accepted,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,"3000 MARKET ST, SAN FRANCISCO, CA, 94114",MARKET ST,8,Eureka Valley,MISSION,37.7611707,-122.4410268,"(37.7611707, -122.4410268)",Web,None
3,12703351,07/28/2020 12:38:00 PM,07/28/2020 01:18:00 PM,07/28/2020 01:18:00 PM,Closed,Case Resolved,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,"1401 UTAH ST, SAN FRANCISCO, CA, 94110",UTAH ST,10,Potrero Hill,MISSION,37.75158593,-122.40507703,"(37.75158593, -122.40507703)",Mobile/Open311,http://mobile311.sfgov.org/reports/12703351/ph...
4,12705318,07/28/2020 06:30:00 PM,None,07/28/2020 06:55:02 PM,Open,in progress,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,"3564 18TH ST, SAN FRANCISCO, CA, 94110",18TH ST,8,Mission Dolores,MISSION,37.761959205528,-122.422836029721,"(37.76195921, -122.42283603)",Mobile/Open311,http://mobile311.sfgov.org/reports/12705318/ph...
5,12704075,07/28/2020 02:27:00 PM,07/28/2020 02:41:48 PM,07/28/2020 02:41:48 PM,Closed,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Boxed or Bagged Items,"1055 YORK ST, SAN FRANCISCO, CA, 94110",YORK ST,9,Mission,MISSION,37.755187988281,-122.408233642578,"(37.75518799, -122.40823364)",Phone,None
6,12702963,07/28/2020 11:40:00 AM,None,07/28/2020 12:00:21 PM,Open,in progress,DPW Ops Queue,Street and Sidewalk Cleaning,Human or Animal Waste,Human or Animal Waste,"223 3RD ST, SAN FRANCISCO, CA, 94103",3RD ST,6,South of Market,SOUTHERN,37.784753758388,-122.400231178049,"(37.78475376, -122.40023118)",Web,None
7,12704112,07/28/2020 02:34:00 PM,None,07/28/2020 03:26:42 PM,Open,sent,Port Authority - G,Street and Sidewalk Cleaning,Human or Animal Waste,Human or Animal Waste,"1 FERRY PLZ, SAN FRANCISCO, CA, 94111",FERRY PLZ,3,Financial District,CENTRAL,37.7952003479,-122.393089294434,"(37.79520035, -122.39308929)",Phone,None
8,12705081,07/28/2020 05:32:18 PM,07/28/2020 06:16:28 PM,07/28/2020 06:16:28 PM,Closed,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Boxed or Bagged Items,"990 BRAZIL AVE, SAN FRANCISCO, CA, 94112",BRAZIL AVE,11,Excelsior,INGLESIDE,37.72088289492,-122.427076668503,"(37.72088289, -122.42707667)",Mobile/Open311,http://mobile311.sfgov.org/reports/12705081/ph...
9,12700983,07/27/2020 08:54:00 PM,None,07/27/2020 09:55:03 PM,Open,in progress,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,"44 LANGTON ST, SAN FRANCISCO, CA, 94103",LANGTON ST,6,South of Market,SOUTHERN,37.7769039,-122.4092373,"(37.7769039, -122.4092373)",Mobile/Open311,http://mobile311.sfgov.org/reports/12700983/ph...


In [8]:
sf.createOrReplaceTempView('table')
query = '''
SELECT
    Opened,
    Closed
FROM
    table
WHERE
    not isnull(Closed)
'''
spark.sql(query).toPandas()

,Opened,Closed
0,07/28/2020 12:38:00 PM,07/28/2020 01:18:00 PM
1,07/28/2020 02:27:00 PM,07/28/2020 02:41:48 PM
2,07/28/2020 05:32:18 PM,07/28/2020 06:16:28 PM
3,07/27/2020 03:11:43 PM,07/28/2020 07:52:30 AM
4,07/27/2020 01:48:53 PM,07/27/2020 02:06:42 PM
...,...,...
9824,07/06/2015 10:58:35 AM,07/07/2015 08:48:43 AM
9825,07/07/2015 04:34:08 PM,07/08/2015 11:00:00 AM
9826,07/07/2015 04:49:57 PM,07/08/2015 10:01:44 AM
9827,07/07/2015 05:40:03 PM,07/08/2015 09:58:57 AM


In [9]:
dt_format = 'MM/dd/yyyy hh:mm:ss a'
sf2 = sf.select(
        fn.to_timestamp('Opened', format=dt_format).alias('Opened'),
        fn.to_timestamp('Closed', format=dt_format).alias('Closed'),
        fn.to_timestamp('Updated', format=dt_format).alias('Updated'),
    ).filter(
        sf.Closed.isNotNull(),
    )
sf2.select((
        sf2.Closed.cast('long') - sf2.Opened.cast('long')
        ).alias('Time_to_resolve')
    ).toPandas()

,Time_to_resolve
0,2400.0
1,888.0
2,2650.0
3,60047.0
4,1069.0
...,...
9824,78608.0
9825,66352.0
9826,61907.0
9827,58734.0


In [10]:
holidays_US = holidays.US(state='CA', years=range(2017, 2021))